In [1]:
%reload_ext autoreload
%autoreload 2

import os, sys
if os.environ['USER'] == 'yitians' and 'submit' in os.uname().nodename:
    os.environ['DM21CM_DATA_DIR'] = '/data/submit/yitians/dm21cm/DM21cm'
    os.environ['DH_DIR'] = '/work/submit/yitians/darkhistory/DarkHistory'
sys.path.append('..')
sys.path.append(os.environ['DH_DIR'])

from tqdm import tqdm
import time
import pickle

import numpy as np
import jax
import jax.numpy as jnp

import py21cmfast as p21c
from py21cmfast import plotting, cache_tools
print(f'Using 21cmFAST version {p21c.__version__}')

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
mpl.rc_file('../matplotlibrc')

/n/home07/yitians/anaconda3/envs/21cmfast/lib/python3.7/site-packages/py21cmfast/_cfg.py:59: UserWarning: Your configuration file is out of date. Updating...
  "Your configuration file is out of date. Updating..."
/n/home07/yitians/anaconda3/envs/21cmfast/lib/python3.7/site-packages/py21cmfast/_cfg.py:42: UserWarning: Your configuration file is out of date. Updating...
  warnings.warn("Your configuration file is out of date. Updating...")


Using 21cmFAST version 0.1.dev1579+g6b1da6d


In [6]:
import logging
logger = logging.getLogger('21cmFAST')
logger.setLevel(logging.INFO)
logging.getLogger('py21cmfast._utils').setLevel(logging.CRITICAL+1)
logging.getLogger('py21cmfast.wrapper').setLevel(logging.CRITICAL+1)

In [9]:
from dm21cm.dm_params import DMParams
from dm21cm.data_loader import load_data
import dm21cm.physics as phys

from darkhistory.spec.spectrum import Spectrum

In [10]:
jax.devices()

[StreamExecutorGpuDevice(id=0, process_index=0, slice_index=0),
 StreamExecutorGpuDevice(id=1, process_index=0, slice_index=0)]

## 0. Global config

In [11]:
! lscpu | grep "CPU(s)"

CPU(s):                64
On-line CPU(s) list:   0-63
NUMA node0 CPU(s):     0-31
NUMA node1 CPU(s):     32-63


In [12]:
N_THREADS = 32

In [13]:
def get_z_arr(z_start=None, z_end=20):
    
    if z_start is None:
        z_start = p21c.global_params.Z_HEAT_MAX
    z_arr = [z_end]
    while np.max(z_arr) < z_start:
        z_prev = (1 + np.max(z_arr)) * p21c.global_params.ZPRIME_STEP_FACTOR - 1
        z_arr.append(z_prev)
    return np.array(z_arr[::-1][1:])

In [24]:
# check cached runs
CACHE_DIR_BASE = os.environ['P21C_CACHE_DIR']
os.listdir(CACHE_DIR_BASE)

['base']

## 1. Initialization

In [25]:
# config
RUN_NAME = 'phph'
struct_boost_model = 'erfc 1e-3' # default: erfc 1e-3
run_mode = 'inj' # '' or 'inj'

dh_init_phot_bath = False
dh_init_T_k = False
enable_phot_bath = True

debug_uniform_injection = False

reload = False
use_tqdm = True
save_slices = True

abscs = pickle.load(open('../data/abscissas/abscs_nBs_test_2.p', 'rb'))

p21c.config['direc'] = f'{CACHE_DIR_BASE}/{RUN_NAME}'
os.makedirs(p21c.config['direc'], exist_ok=True)

In [26]:
cache_tools.clear_cache()

2023-04-15 23:48:31,749 | INFO | Removed 0 files from cache.
INFO:21cmFAST:Removed 0 files from cache.


## 2. Loop

In [27]:
# redshift
p21c.global_params.ZPRIME_STEP_FACTOR = 1.05
p21c.global_params.Z_HEAT_MAX = 44.
z_arr = get_z_arr(z_end=6.)
#p21c.global_params.CLUMPING_FACTOR = 1.

# dark matter
dm_params = DMParams(mode='swave', primary='phot_delta', m_DM=1e10, sigmav=1e-23)

if dh_init_phot_bath or dh_init_T_k:
    
    dhinit_fn = f"{p21c.config['direc']}/dhinit_soln.p"
    
    if os.path.exists(dhinit_fn):
        dhinit_soln = pickle.load(open(dhinit_fn, 'rb'))
    else:
        logger.info('Running DarkHistory to generate initial conditions.')
        
        import main
        dhinit_soln = main.evolve(
            DM_process=dm_params.mode, mDM=dm_params.m_DM,
            sigmav=dm_params.sigmav, primary=dm_params.primary,
            struct_boost=phys.struct_boost_func(model=struct_boost_model),
            start_rs=3000, end_rs=z_arr[0], coarsen_factor=12, verbose=1
        )
        pickle.dump(dhinit_soln, open(dhinit_fn, 'wb'))

In [28]:
# physics
cosmo_params = dict(OMm=0.32, OMb=0.049, POWER_INDEX=0.96, SIGMA_8=0.83, hlittle=0.67)

initial_conditions = p21c.initial_conditions(
    user_params = p21c.UserParams(
        HII_DIM = 50, # [1]
        BOX_LEN = 50, # [p-Mpc]
        N_THREADS = N_THREADS
    ),
    cosmo_params = p21c.CosmoParams(**cosmo_params),
    random_seed=54321, write=True
)

# box
box_dim = initial_conditions.user_params.HII_DIM

# recording
records = []
input_time_tot = 0.
p21c_time_tot = 0.
if use_tqdm:
    pbar = tqdm(total=len(z_arr))
    
if save_slices:
    saved_slices = []
    i_slice = int(box_dim/2)

## LOOP
for i_z in range(len(z_arr)):

    z = z_arr[i_z]
    
    input_timer = time.time()
    
    if i_z == 0:
        spin_temp = None
        input_heating = input_ionization = input_jalpha = None
        
        ## load tfs
        phot_phot_tf = load_data('phot_phot', reload=reload)
        elec_phot_tf = load_data('elec_phot', reload=reload)
        phot_dep_tf = load_data('phot_dep', reload=reload)
        elec_dep_tf = load_data('elec_dep', reload=reload)
        
        ## fix DM in spec
        phot_phot_tf.set_fixed_in_spec(dm_params.inj_phot_spec.N)
        elec_phot_tf.set_fixed_in_spec(dm_params.inj_elec_spec.N)
        phot_dep_tf.set_fixed_in_spec(dm_params.inj_phot_spec.N)
        elec_dep_tf.set_fixed_in_spec(dm_params.inj_elec_spec.N)
        
        ## photon bath
        if dh_init_phot_bath:
            phot_bath_next = dhinit_soln['highengphot'][-1]
        else:
            phot_bath_next = Spectrum(
                abscs['photE'], np.zeros_like(abscs['photE']), spec_type='N', rs=1+z_arr[0]
            ) # [N per Bavg]
    
    else: ## input from second step
        
        input_heating = p21c.input_heating(redshift=z, init_boxes=initial_conditions, write=False)
        input_ionization = p21c.input_ionization(redshift=z, init_boxes=initial_conditions, write=False)
        input_jalpha = p21c.input_jalpha(redshift=z, init_boxes=initial_conditions, write=False)
        
        if i_z == 1 and dh_init_T_k:
            T_k_21cmfast = np.mean(spin_temp.Tk_box)
            T_k_DH = dhinit_soln['Tm'][-1] / phys.kB
            input_heating.input_heating += (T_k_DH - T_k_21cmfast)

        if run_mode == '':
            if i_z == 1:
                logger.warning('Not injecting anything in this run!')

        elif run_mode == 'inj':
            
            ## common
            delta_box = jnp.asarray(perturbed_field.density)
            x_e_box = jnp.asarray(1 - ionized_box.xH_box)
            B_per_Bavg = 1 + delta_box
            
            if debug_uniform_injection:
                delta_box = jnp.full_like(delta_box, jnp.mean(delta_box))
                x_e_box = jnp.full_like(x_e_box, jnp.mean(x_e_box))
                B_per_Bavg = 1 + delta_box
            
            ## photon bath: redshift from previous step
            if enable_phot_bath:
                phot_bath_next.redshift(1+z)
                phot_bath = phot_bath_next
            
            ## photon bath -> photon
            if enable_phot_bath:
                out_phot_N = phot_phot_tf(
                    rs = 1 + z,
                    in_spec = phot_bath.N,
                    nBs_s = (1+delta_box).ravel(),
                    x_s = x_e_box.ravel(),
                    sum_result = True,
                    out_of_bounds_action = 'clip',
                ) / (box_dim ** 3) # [N / Bavg]

                phot_bath_next = Spectrum(
                    phot_phot_tf.abscs['out'],
                    out_phot_N,
                    spec_type='N', rs=1+z
                ) # [N / Bavg]
            
            ## photon bath -> deposition
            if enable_phot_bath:
                phot_bath_dep_box = phot_dep_tf(
                    rs = 1 + z,
                    in_spec = phot_bath.N,
                    nBs_s = (1+delta_box).ravel(),
                    x_s = x_e_box.ravel(),
                    sum_result = False,
                    out_of_bounds_action = 'clip',
                ).reshape(box_dim, box_dim, box_dim, len(abscs['dep_c'])) # [eV / Bavg]
            else:
                phot_bath_dep_box = np.zeros((box_dim, box_dim, box_dim, len(abscs['dep_c']))) # [eV / Bavg]
            
            ## DM: common
            z_prev = z_arr[i_z-1]
            dt = phys.dt_between_z(z_prev, z) # [s]
            if dm_params.mode == 'swave':
                struct_boost = phys.struct_boost_func(model=struct_boost_model)(1+z)
            else:
                struct_boost = 1
            
            rho_DM_box = (1 + delta_box) * phys.rho_DM * (1+z)**3 # [eV cm^-3]
            n_Bavg = phys.n_B * (1+z)**3 # [Bavg cm^-3]
            inj_per_Bavg_box = phys.inj_rate(rho_DM_box, dm_params) * dt * struct_boost / n_Bavg # [inj/Bavg]
            
            ## DM -> photon
            if enable_phot_bath:
                out_phot_N_phot = phot_phot_tf(
                    rs = 1 + z,
                    in_spec = dm_params.inj_phot_spec.N,
                    nBs_s = (1+delta_box).ravel(),
                    x_s = x_e_box.ravel(),
                    sum_result = True,
                    sum_weight = inj_per_Bavg_box.ravel(),
                    out_of_bounds_action = 'clip',
                ) / (box_dim ** 3) # [N / Bavg]

                out_phot_N_elec = elec_phot_tf(
                    rs = 1 + z,
                    in_spec = dm_params.inj_elec_spec.N,
                    nBs_s = (1+delta_box).ravel(),
                    x_s = x_e_box.ravel(),
                    sum_result = True,
                    sum_weight = inj_per_Bavg_box.ravel(),
                    out_of_bounds_action = 'clip',
                ) / (box_dim ** 3) # [N / Bavg]

                phot_bath_next += Spectrum(
                    phot_phot_tf.abscs['out'],
                    out_phot_N_phot + out_phot_N_elec,
                    spec_type='N', rs=1+z
                ) # [N / Bavg]
            
            ## DM -> deposition
            DM_phot_dep_box = phot_dep_tf(
                rs = 1 + z,
                in_spec = dm_params.inj_phot_spec.N,
                nBs_s = (1+delta_box).ravel(),
                x_s = x_e_box.ravel(),
                sum_result = False,
                out_of_bounds_action = 'clip',
            ).reshape(box_dim, box_dim, box_dim, len(abscs['dep_c'])) * inj_per_Bavg_box[...,None] # [eV / Bavg]

            DM_elec_dep_box = elec_dep_tf(
                rs = 1 + z,
                in_spec = dm_params.inj_elec_spec.N,
                nBs_s = (1+delta_box).ravel(),
                x_s = x_e_box.ravel(),
                sum_result = False,
                out_of_bounds_action = 'clip',
            ).reshape(box_dim, box_dim, box_dim, len(abscs['dep_c'])) * inj_per_Bavg_box[...,None] # [eV / Bavg]
            
            ## deposite energy
            dep_box = phot_bath_dep_box + DM_phot_dep_box + DM_elec_dep_box # [eV / Bavg]
            # last dimension: ('H ion', 'He ion', 'exc', 'heat', 'cont')

            input_heating.input_heating += np.array(
                2 / (3*phys.kB*(1+x_e_box)) * dep_box[...,3] / B_per_Bavg
            ) # [K/Bavg] / [B/Bavg] = [K/B]
            input_ionization.input_ionization += np.array(
                (dep_box[...,0] + dep_box[...,1]) / phys.rydberg / B_per_Bavg
            ) # [1/Bavg] / [B/Bavg] = [1/B]
            
            n_lya = dep_box[...,2] * n_Bavg / phys.lya_eng # [lya cm^-3]
            dnu_lya = (phys.rydberg - phys.lya_eng) / (2*np.pi*phys.hbar) # [Hz^-1]
            J_lya = n_lya * phys.c / (4*np.pi) / dnu_lya # [lya cm^-2 s^-1 sr^-1 Hz^-1]
            input_jalpha.input_jalpha += np.array( J_lya )
            
            ## record
            dE_inj_per_Bavg = dm_params.eng_per_inj * np.mean(inj_per_Bavg_box) # [eV per Bavg]
            dE_inj_per_Bavg_unclustered = dE_inj_per_Bavg / struct_boost
            record_inj = {
                'dE_inj_per_B' : dE_inj_per_Bavg,
                'f_heat' : np.mean(dep_box[...,3]) / dE_inj_per_Bavg_unclustered,
                'f_ion'  : np.mean(dep_box[...,0] + dep_box[...,1]) / dE_inj_per_Bavg_unclustered,
                'f_exc'  : np.mean(dep_box[...,2]) / dE_inj_per_Bavg_unclustered,
            }
            
        else:
            raise ValueError(run_mode)
            
    input_time_tot += time.time() - input_timer

    ## step in 21cmFAST
    p21c_timer = time.time()
    perturbed_field = p21c.perturb_field( # perturbed_field controls the redshift
        redshift=z,
        init_boxes=initial_conditions
    )
    spin_temp = p21c.spin_temperature(
        perturbed_field=perturbed_field,
        previous_spin_temp=spin_temp,
        input_heating_box=input_heating,
        input_ionization_box=input_ionization,
        input_jalpha_box=input_jalpha,
        write=True
    )
    ionized_box = p21c.ionize_box(
        spin_temp=spin_temp
    )
    brightness_temp = p21c.brightness_temperature(
        ionized_box=ionized_box,
        perturbed_field=perturbed_field,
        spin_temp=spin_temp
    )
    p21c_time_tot += time.time() - p21c_timer

    ## save results
    if i_z > 0:
        record = {
            'z'   : z,
            'T_s' : np.mean(spin_temp.Ts_box), # [mK]
            'T_b' : np.mean(brightness_temp.brightness_temp), # [K]
            'T_k' : np.mean(spin_temp.Tk_box), # [K]
            'x_e' : np.mean(1 - ionized_box.xH_box), # [1]
        }
        if run_mode == 'inj':
            record.update(record_inj)
        records.append(record)
        
    if save_slices:
        saved_slices.append({
            'z'   : z,
            'T_s' : spin_temp.Ts_box[i_slice], # [mK]
            'T_b' : brightness_temp.brightness_temp[i_slice], # [K]
            'T_k' : spin_temp.Tk_box[i_slice], # [K]
            'x_e' : 1 - ionized_box.xH_box[i_slice], # [1]
            'delta' : perturbed_field.density[i_slice], # [1]
        })
        
    if use_tqdm:
        pbar.update()
    
print(f'input used {input_time_tot:.4f} s')
print(f'p21c used {p21c_time_tot:.4f} s')

arr_records = {k: np.array([r[k] for r in records]) for k in records[0].keys()}
np.save(f'../data/run_info/{RUN_NAME}_records', arr_records)
if save_slices:
    np.save(f'../data/run_info/{RUN_NAME}_slices', saved_slices)


100%|███████████████████████████████████████████████████████████████████████████████████| 39/39 [09:25<00:00, 14.50s/it]

100%|███████████████████████████████████████████████████████████████████████████████████| 39/39 [03:20<00:00,  4.68s/it]

input used 97.6443 s
p21c used 103.1387 s
